In [38]:
import sys
import os

# Add notebooks directory to Python path
notebook_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if notebook_dir not in sys.path:
    sys.path.append(notebook_dir)

from test_utils import run_tests

## Similarity Algorithms

The similarity algorithms available in `textdistance` package are given in the tables below.

### Edit Distance

| Algorithm            | Description                                                                 |
|----------------------|-----------------------------------------------------------------------------|
| [`damerau_levenshtein`](https://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance) | Similar to Levenshtein but considers transpositions as a single edit. |
| [`hamming`](https://en.wikipedia.org/wiki/Hamming_distance)            | Measures the number of positions at which the corresponding symbols are different. |
| [`levenshtein`](https://en.wikipedia.org/wiki/Levenshtein_distance)        | Calculates the minimum number of single-character edits required to change one word into the other. |
| [`jaro`](https://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance)               | Measures similarity between two strings, giving more weight to common prefixes. |
| [`jaro_winkler`](https://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance)       | An extension of Jaro, giving more weight to strings that match from the beginning. |
| [`lcsseq`](https://en.wikipedia.org/wiki/Longest_common_subsequence_problem)             | Measures the longest common subsequence. |
| [`lcsstr`](https://docs.python.org/2/library/difflib.html#difflib.SequenceMatcher)             | Measures the longest common substring. |
| [`ratcliff_obershelp`](https://en.wikipedia.org/wiki/Gestalt_Pattern_Matching) | Measures similarity based on the longest common subsequence. |
| [`strcmp95`](http://cpansearch.perl.org/src/SCW/Text-JaroWinkler-0.1/strcmp95.c)           | A string comparison algorithm developed by the U.S. Census Bureau. |
| [`needleman_wunsch`](https://en.wikipedia.org/wiki/Needleman%E2%80%93Wunsch_algorithm)   | A dynamic programming algorithm for sequence alignment. |
| [`smith_waterman`](https://en.wikipedia.org/wiki/Smith%E2%80%93Waterman_algorithm)     | A dynamic programming algorithm for local sequence alignment. |
| [`gotoh`](http://bioinfo.ict.ac.cn/~dbu/AlgorithmCourses/Lectures/LOA/Lec6-Sequence-Alignment-Affine-Gaps-Gotoh1982.pdf)              | An extension of Needleman-Wunsch with affine gap penalties. |

### Token

| Algorithm            | Description                                                                 |
|----------------------|-----------------------------------------------------------------------------|
| [`cosine`](https://en.wikipedia.org/wiki/Cosine_similarity)             | Measures the cosine of the angle between two non-zero vectors. |
| [`jaccard`](https://en.wikipedia.org/wiki/Jaccard_index)            | Measures similarity between finite sample sets. |
| [`overlap`](https://en.wikipedia.org/wiki/Overlap_coefficient)            | Measures the overlap coefficient between two sets. |
| [`sorensen`](https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient)           | Measures similarity between two sets, based on the size of the intersection divided by the size of the union. |
| [`sorensen_dice`](https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient)      | Similar to Sorensen, but uses Dice's coefficient. |
| [`dice`](https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient)               | Another name for Sorensen-Dice coefficient. |
| [`tversky`](https://en.wikipedia.org/wiki/Tversky_index)            | A generalization of the Jaccard index. |

### Sequence

| Algorithm            | Description                                                                 |
|----------------------|-----------------------------------------------------------------------------|
| [`bag`](https://github.com/Yomguithereal/talisman/blob/master/src/metrics/bag.js)                | Measures bag similarity between two sequences                               |
| [`mlipns`](http://www.sial.iias.spb.su/files/386-386-1-PB.pdf)             | Measures similarity using the MLIPNS algorithm                              |
| [`monge_elkan`](https://www.academia.edu/200314/Generalized_Monge-Elkan_Method_for_Approximate_Text_String_Comparison)        | A hybrid algorithm combining multiple similarity measures. $ME(a,b)$ |

### Phonetic

| Algorithm                                                                    | Description                                                                 |
|------------------------------------------------------------------------------|-----------------------------------------------------------------------------|
| [`mra`](https://en.wikipedia.org/wiki/Match_rating_approach)                 | Measures similarity using the MRA algorithm                                 |
| [`editex`](https://anhaidgroup.github.io/py_stringmatching/v0.3.x/Editex.html) | Measures similarity using the Editex algorithm                              |                              |

In [39]:
import textdistance

def text_distance(needle, haystack, algorithm):
    """
    Calculate similarity scores between search term(s) and a list of strings.
    
    Args:
        needle (str|list): Single string or 2D list containing search term(s)
        haystack (list): 2D list as column vector, e.g. [['apple'], ['banana']]
        algorithm (str): Text distance algorithm name from textdistance library
    
    Returns:
        list[list]: 2D list of [index, score] pairs, where index is 1-based position
            in haystack and score is normalized similarity (0-1 scale).
    """
    algo_func = getattr(textdistance, algorithm)
    
    # Handle both string and 2D list input for needle
    needle_list = [needle] if isinstance(needle, str) else [item for sublist in needle for item in sublist]
    
    results = []
    for needle_item in needle_list:
        scores = [(index + 1, round(algo_func.normalized_similarity(needle_item, item[0]), 2)) 
                 for index, item in enumerate(haystack)]
        scores.sort(key=lambda x: x[1], reverse=True)
        results.append(list(scores[0]))

    return results

# Test cases with column vectors
test_haystack = [['apple'], ['banana'], ['orange'], ['pear']]
test_needle_2d = [['aple'], ['banan']]

test_cases = [
    ['aple', test_haystack, 'jaccard'],
    ['banan', test_haystack, 'levenshtein'],
    [test_needle_2d, test_haystack, 'jaccard'],
    ['peer', test_haystack, 'jaro_winkler']
]

run_tests(text_distance, test_cases)


Case 1: ['aple', [['apple'], ['banana'], ['orange'], ['pear']], 'jaccard'] -> [[1, 0.8]]
Case 2: ['banan', [['apple'], ['banana'], ['orange'], ['pear']], 'levenshtein'] -> [[2, 0.83]]
Case 3: [[['aple'], ['banan']], [['apple'], ['banana'], ['orange'], ['pear']], 'jaccard'] -> [[1, 0.8], [2, 0.83]]
Case 4: ['peer', [['apple'], ['banana'], ['orange'], ['pear']], 'jaro_winkler'] -> [[4, 0.87]]


In [40]:
import textdistance

def fuzzy_top_n(needle, haystack, algorithm, top_n):
    """
    Find top N most similar strings for given search term(s).
    
    Args:
        needle (str|list): Single string or 2D list containing search term(s)
        haystack (list): 2D list as column vector, e.g. [['apple'], ['banana']]
        algorithm (str): Text distance algorithm name from textdistance library
        top_n (int): Number of closest matches to return for each needle
    
    Returns:
        list[list]: 2D list where each inner list contains top N matching strings
    """
    algo_func = getattr(textdistance, algorithm)
    
    needle_list = [needle] if isinstance(needle, str) else [item for sublist in needle for item in sublist]
    
    results = []
    for needle_item in needle_list:
        scores = [(item[0], round(algo_func.normalized_similarity(needle_item, item[0]), 2))
                 for item in haystack]
        scores.sort(key=lambda x: x[1], reverse=True)
        top_matches = [score[0] for score in scores[:top_n]]
        results.append(top_matches)
    
    return [results[0]] if len(results) == 1 else results

# Test cases with column vectors
test_haystack = [['apple'], ['banana'], ['orange'], ['pear'], ['apricot'], ['grape']]
test_needle_2d = [['aple'], ['banan'], ['oragne']]

test_cases = [
    ['aple', test_haystack, 'jaccard', 2],
    ['oragne', test_haystack, 'levenshtein', 3],
    [test_needle_2d, test_haystack, 'jaro_winkler', 2],
    ['peer', test_haystack, 'hamming', 3]
]

run_tests(fuzzy_top_n, test_cases)


Case 1: ['aple', [['apple'], ['banana'], ['orange'], ['pear'], ['apricot'], ['grape']], 'jaccard', 2] -> [['apple', 'pear']]
Case 2: ['oragne', [['apple'], ['banana'], ['orange'], ['pear'], ['apricot'], ['grape']], 'levenshtein', 3] -> [['orange', 'grape', 'apple']]
Case 3: [[['aple'], ['banan'], ['oragne']], [['apple'], ['banana'], ['orange'], ['pear'], ['apricot'], ['grape']], 'jaro_winkler', 2] -> [['apple', 'orange'], ['banana', 'orange'], ['orange', 'grape']]
Case 4: ['peer', [['apple'], ['banana'], ['orange'], ['pear'], ['apricot'], ['grape']], 'hamming', 3] -> [['pear', 'apple', 'banana']]


In [41]:
import textdistance

def fuzzy_matcher(needle, haystack):
    """
    Find top 3 most similar strings for given search term(s) using Jaccard algorithm.
    
    Args:
        needle (str|list): Single string or 2D list containing search term(s)
        haystack (list): 2D list as column vector, e.g. [['apple'], ['banana']]
    
    Returns:
        list[list]: 2D list where each inner list contains top 3 matching strings
    """
    algo_func = textdistance.jaccard
    
    needle_list = [needle] if isinstance(needle, str) else [item for sublist in needle for item in sublist]
    
    results = []
    for needle_item in needle_list:
        scores = [(item[0], round(algo_func.normalized_similarity(needle_item, item[0]), 2))
                 for item in haystack]
        scores.sort(key=lambda x: x[1], reverse=True)
        top_matches = [score[0] for score in scores[:3]]
        results.append(top_matches)
    
    return [results[0]] if len(results) == 1 else results

# Test cases with column vectors
test_haystack = [['apple'], ['banana'], ['orange'], ['pear'], ['apricot'], ['grape']]
test_needle_2d = [['aple'], ['banan'], ['oragne']]

test_cases = [
    ['aple', test_haystack],
    ['oragne', test_haystack],
    [test_needle_2d, test_haystack],
    ['peer', test_haystack]
]

run_tests(fuzzy_matcher, test_cases)

Case 1: ['aple', [['apple'], ['banana'], ['orange'], ['pear'], ['apricot'], ['grape']]] -> [['apple', 'pear', 'grape']]
Case 2: ['oragne', [['apple'], ['banana'], ['orange'], ['pear'], ['apricot'], ['grape']]] -> [['orange', 'grape', 'pear']]
Case 3: [[['aple'], ['banan'], ['oragne']], [['apple'], ['banana'], ['orange'], ['pear'], ['apricot'], ['grape']]] -> [['apple', 'pear', 'grape'], ['banana', 'orange', 'pear'], ['orange', 'grape', 'pear']]
Case 4: ['peer', [['apple'], ['banana'], ['orange'], ['pear'], ['apricot'], ['grape']]] -> [['pear', 'grape', 'apple']]
